## Test AI Angent Work flow for Bank Statements

In [68]:
import os, json, time, gc

from dotenv import load_dotenv
from IPython.display import HTML, Markdown, Image, Video
from tqdm import tqdm
from openai import OpenAI, AsyncOpenAI
import asyncio
import aiohttp
import pandas as pd
import torch

#fix bug with aysncio and jupyter
import nest_asyncio # for langchain async 
nest_asyncio.apply()

import numpy as np

In [69]:
class CFG:
    OFFLINE = False #True # for Test offline environment
    USE_LLAMA3 = False # 
    USE_GEMMA2 = False # 
    USE_QWEN = False # 
    USE_DEEPSEEK = True # 
    USE_DEEPSCALE = False # 

    TASK_GEN = True # for generative Text output task (suitable for RAG project)
    TEST_LLM = True
    USE_HUGGINGFACE = True # Pull model from Huggingface model hub
    USE_LMSTUIDO = False # for local LLM framework 
    USE_OLLAMA = False # for OLLAMA local LLM framework 
    USE_VLLM = False # for VLLM  LLM framework

    # mulitlingual LLM model 
    model1 = "meta-llama/Llama-3.2-3B-Instruct"  # llama3.2  3B-Instruct

    model2 =  "google/gemma-2-2b-it" # gemma 2 9B (mulitlingual)
    model3 = "Qwen/Qwen2.5-3B-Instruct" # Qwen 3B (mulitlingual)
    model4 = 'Qwen/Qwen2.5-7B-Instruct' # Qwen 7B (mulitlingual)
    model5 = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B" # DeepSeek Distill 1.5B (mulitlingual)
    model6 = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B" # DeepSeek Distill 7B (mulitlingual)
    model7 = "agentica-org/DeepScaleR-1.5B-Preview"

    # for VLM model
    vlmModel1 = "Qwen/Qwen2.5-VL-3B-Instruct"
    vlmModel2 = "Qwen/Qwen2.5-VL-7B-Instruct"


    # Mult Embedding model
    embedModel1 = 'intfloat/multilingual-e5-small' # for embedding model support chinese
    embedModel2 = "intfloat/multilingual-e5-large-instruct"
    embedModel3 = "Alibaba-NLP/gte-Qwen2-1.5B-instruct" # for embedding model support chinese
    embedModel4 = "Alibaba-NLP/gte-multilingual-base" # for embedding model support chinese
    embedModel5 = "BAAI/bge-m3" # for multilingual embedding model
    embedModel6 = "jinaai/jina-embeddings-v3"
    

    # LLM for AI Agent 
    llmModel1 = "ollama/deepseek-r1:8b"
    llmModel2 = "ollama/deepseek-r1:7b"
    llm_base_url1 = "http://localhost:11434"

    # 



    FEW_SHOT_TEST= False#True
    USE_WANDB = True#True # for  LLM evalution and debug , track fine tuning performance

    USE_DEEPEVAL = True#False # for LLM evalution   
    USE_TRAIN =  False #True #False#True Much be use GPU for Training 
    
    # For VectorDB selection
    USE_FAISS = False#True # For RAG VectorDB
    USE_CHROMA = True #False #True #False # for RAG VectorDF
    USE_PINECONE = False#True#False #True # for RAG VectorDF
    USE_WEAVIATE = False#True #False # for RAG VectorDF
    USE_MILVUS = False#True              # for RAG VectorDF

    # for LLM fine tuning
    maxTrainData = 200#3500#5000 #10000#5000 #10000
    maxEvalData = 20#100 # 20 


    # LLM parameters
    reportTo ="none"
    topK = 40
    topP = 1.0
    temperature = 0.6 #0.5
    repetition_penalty = 1.05 # 1.1
    maxOutToken = 1024#180 #100
    

    
    maxToken=  512#768#512#768 # 512 for test only

In [70]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import ScrapeWebsiteTool, SerperDevTool
from crewai.tools import BaseTool, tool
from crewai import LLM
import crewai
from pydantic import BaseModel, Field
from typing import List, Dict, Type
from crewai_tools import PDFSearchTool

In [71]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [72]:
from langchain_ollama import OllamaLLM, ChatOllama
from langchain_openai import ChatOpenAI


In [73]:
import ctypes
def clearMemory():
    for _ in range(5):
        torch.cuda.empty_cache()
        ctypes.CDLL("libc.so.6").malloc_trim(0)
        gc.collect()
        time.sleep(0.3)

In [74]:
clearMemory()

In [75]:
## Define Test Document path 
pdfFilePath1 = "../test-document/Attention .pdf"
pdfFilePath2 = "../test-document/yolo.pdf"
pdfDir = "../test-document"

bankStatementDir = "../bank-statement-document/"
bankStatementSamples =  "../bank-statement-document/Bank-Statement-Template-2-TemplateLab.pdf"

## CrewAI Support LLM List :  LiteLLM 
- <https://docs.crewai.com/how-to/llm-connections>
- <https://docs.crewai.com/concepts/llms>



In [76]:
from crewai_tools import PDFSearchTool, YoutubeChannelSearchTool
from crewai_tools import RagTool

from crewai.tools import BaseTool, tool

# Chanin
from langchain_community.document_loaders import (TextLoader,
                                                  PyMuPDFLoader,
                                                  PyPDFDirectoryLoader,
                                                  PyPDFLoader)

# Define the tools for CrewAI agent

In [77]:
#Tool for AI agent

# pdfTool = PDFSearchTool(pdfDir=bankStatementDir)

In [134]:
# Custom PDF Search Tool for CrewAI
@tool("Custom PDF Extractor")
def pdf_extractor(
    pdf_dir: str
) -> list:
    """
    Extracts text from PDF files in the specified directory.
    Args:
        pdf_dir (str): The directory containing PDF files.
    Returns:
        List[Dict[str, str]]: A list of dictionaries containing the extracted text from each PDF file.
    """
    # results = pdfTool.search(query=query, pdf_dir=pdf_dir)
    loader = PyMuPDFLoader(pdf_dir)
    documents = loader.load()
    return documents

In [135]:
@tool("PDF directory Extractor")
def pdf_directory_extractor(
    pdf_dir: str = pdfDir
) -> list:
    """
    Extracts text from all PDF files in the specified directory.
    Parameters:
    - pdf_dir (str): The directory containing PDF files.
    Returns:
    - List of dictionaries containing the content and source of each PDF file.
    """
    loader = PyPDFDirectoryLoader(pdf_dir)
    documents = loader.load()
    
    # results = []
    # for doc in documents:
    #     if query.lower() in doc.page_content.lower():
    #         results.append({"content": doc.page_content, "source": doc.metadata.get("source", "")})
    
    # return results

    return documents

In [136]:
llm = LLM(
    model= CFG.llmModel2,
    base_url= CFG.llm_base_url1,
)

## Define AI Agent

In [137]:
# 
document_agent = Agent( role="Document Processing Specialist",
    goal="Extract information from a document",
    backstory="You are a document extraction agent. You will be given a document and you need to extract the relevant information from it.",
    # tools=[webSearchTool],
    tools=[pdf_extractor], # Custom PDF Search Tool]
    llm=llm,
    max_iter=10,
    max_execution_time=120, # seconds
    allow_delegation= False,
    verbose=True,

)


In [138]:
data_structuring_agent = Agent(
    role='Data Structuring Expert',
    goal='Parse and structure extracted text into a usable transaction format.',
    backstory="Expert in financial data organization.",
    # tools=[text_tool]
    llm=llm,
    max_iter=10,
    max_execution_time=120, # seconds
    allow_delegation= False,
    verbose=True,
)


In [139]:
financial_analyst_agent = Agent(
    role='Financial Analyst',
    goal='Analyze transaction data and identify key financial insights.',
    backstory="Experienced financial analyst.",
    verbose=True,
    # tools=[analysis_tool]
)

In [140]:
# rag_agent = Agent( role="RAG Agent",
#                   goal="Extract information from a document",
#                  backstory=("You are a document retrieval agent."
#                               "You will be given a document and you need to extract the relevant information from it."),
#                  llm=llm,
#                  allow_delegation= True,
#                  verbose=True,
# )

In [141]:
# fiancial_agent = Agent( role= "Personal Finance Analystic Expert",
#                         goal="Extract information from a document",
#                         backstory=("You are a document retrieval agent."
#                                    "You will be given a document and you need to extract the relevant information from it."),
#                         # tools=[webSearchTool],
#                         llm=llm,
#                         max_iter=10,
#                         allow_delegation= True,
#                         verbose=True,
# )

In [142]:
# summary_agent = Agent( role="Summarization Agent",
#                       goal="Summarize the document",
#                         backstory="You are a document summarization agent. You will be given a document and you need to summarize it.",

# )

In [143]:
report_agent = Agent( role= "Report Writter Expert for Financial Analystic",
                      goal="Generate Clear and Concise Report from the analysis results",
                      backstory=("Professional report writer with expertise in financial analysis."),
                     
                     )





# Task for AI Agent

In [160]:
load_document_task = Task(
    description="load the document from directory:  {pdf_dir} ",
    expected_output=("Output JSON format of the extracted text from the PDF files in the directory with key: content, source , page_number"),
    agent=document_agent,  
)

In [161]:
retrieve_content_task = Task(
    description="Retrieve the content from the document",
    expected_output="Output the content of the document",
)

In [162]:
Financial_Analytic_task = Task(
    description="Extract the financial information from the document, ",
    expected_output="Output the financial information from the document",
)

In [163]:
summarize_task = Task(

    description="Summarize the document",
    expected_output="Output the summary of the document",
)

In [164]:
report_task = Task(

    description="Generate a report from the document",
    expected_output="Output the report of the document",
)

In [165]:
crew1 = Crew(
    tasks=[
        load_document_task,
        # retrieve_content_task,
        # Financial_Analytic_task,
        # summarize_task,
        # report_task
    ],
    agents=[
        document_agent,
        # data_structuring_agent,
        # financial_analyst_agent,
        # rag_agent,
        # fiancial_agent,
        # summary_agent,
        report_agent
    ],
    max_concurrent_tasks=2, # limit the number of concurrent tasks
    verbose=True, 
)


In [166]:
inputs = {
    "pdf_dir" : bankStatementSamples

}

In [167]:
result = await crew1.kickoff_async(inputs= inputs) 

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 7a890fe4-8330-43a7-bc08-a7f7ec0ad5a7                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 0f273e59-53e7-46a6-9667-15951fa8237f
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 0f273e59-53e7-46a6-9667-15951fa8237f
       Status: Executing Task...
    └── 🤖 Agent: Document Processing Specialist
            Status: In Progress

# Agent: Document Processing Specialist
## Task: load the document from directory:  ../bank-statement-document/Bank-Statement-Template-2-TemplateLab.pdf 


🚀 Crew: crew
└── 📋 Task: 0f273e59-53e7-46a6-9667-15951fa8237f
       Status: Executing Task...
    └── 🤖 Agent: Document Processing Specialist
            Status: In Progress
        └── 🧠 Thinking...

...



# Agent: Document Processing Specialist
## Final Answer: 
```json
[
  {
    "content": "The content of the bank statement document goes here.",
    "source": "../bank-statement-document/Bank-Statement-Template-2-TemplateLab.pdf",
    "page_number": 1
  }
]
```




🚀 Crew: crew
└── 📋 Task: 0f273e59-53e7-46a6-9667-15951fa8237f
       Status: Executing Task...
    └── 🤖 Agent: Document Processing Specialist
            Status: ✅ Completed
        └── 🔧 Using Custom PDF Extractor (33)

🚀 Crew: crew
└── 📋 Task: 0f273e59-53e7-46a6-9667-15951fa8237f
       Assigned to: Document Processing Specialist
       Status: ✅ Completed
    └── 🤖 Agent: Document Processing Specialist
            Status: ✅ Completed
        └── 🔧 Using Custom PDF Extractor (33)

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 0f273e59-53e7-46a6-9667-15951fa8237f                                                                     │
│  Agent: Document Processing Specialist                                                                          │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 7a890fe4-8330-43a7-bc08-a7f7ec0ad5a7                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [175]:
print(result.raw)

```json
[
  {
    "content": "The content of the bank statement document goes here.",
    "source": "../bank-statement-document/Bank-Statement-Template-2-TemplateLab.pdf",
    "page_number": 1
  }
]
```


In [ ]:
# print(jsontxt)

"```json\n[\n  {\n    \"content\": \"The content of the bank statement document goes here.\",\n    \"source\": \"../bank-statement-document/Bank-Statement-Template-2-TemplateLab.pdf\",\n    \"page_number\": 1\n  }\n]\n```"
